In [1]:
import yaml
import torch
import numpy as np
from pathlib import Path
from scripts.utils import DataLoader, nanstd
from models.transformer import Transformer
import pandas as pd
import os
from tqdm import tqdm

WORK_PATH = Path.cwd()
FILE_PATH = WORK_PATH / Path('configs/transformer.yml')
print(FILE_PATH)
with open(FILE_PATH, 'r') as file:
    args = yaml.safe_load(file)
dl = DataLoader(args, 'cpu')
model = Transformer('try',dl, args)

/Users/panjianming/学习资料/Berkeley Student Information/Final Trem/AFP
/Users/panjianming/学习资料/Berkeley Student Information/Final Trem/AFP/configs/transformer.yml


In [3]:
# Data
dl.feature.shape

(10, 8654, 1170)

In [20]:
# dl.date
dl.label.shape

(8654, 1170)

In [15]:
K, N, T = dl.feature.shape
df = pd.DataFrame(np.swapaxes(dl.feature, 0, 2).reshape(N*T, K))
df.index = pd.MultiIndex.from_product([dl.date, dl.stock_id], names=['date', 'id'])
df

0         1         2         3         4  \
date       id                                                            
1999-12-31 001004.01  0.497136  0.314126 -0.224952 -0.195079 -0.249172   
           001013.01       NaN       NaN -1.042410 -0.604323  0.604667   
           001034.01  1.560239  1.988937 -0.398602  0.772940  0.023915   
           001038.01  0.189679  0.201219  0.145677 -0.354629 -0.633809   
           001045.01 -0.604194 -0.339149 -0.561381  0.241596 -0.568007   
...                        ...       ...       ...       ...       ...   
2022-05-27 062072.01       NaN       NaN       NaN       NaN       NaN   
           062723.01  1.664235  0.523667 -1.492921 -0.660958 -1.165288   
           106486.01 -3.119771 -1.410208  0.226814 -0.646310 -0.408833   
           164471.02       NaN       NaN       NaN       NaN       NaN   
           178518.02 -0.944431  0.033966 -0.733294  0.791969  0.000000   

                             5         6         7         8    9  
date       id                                                      
1999-12-31 001004.01       NaN  0.177420 -0.273508       NaN  NaN  
           001013.01       NaN -0.023289 -0.584694       NaN  NaN  
           001034.01       NaN -0.157340  0.388202       NaN  NaN  
           001038.01       NaN -0.293197  0.228812       NaN  NaN  
           001045.01       NaN -0.417884  0.387351       NaN  NaN  
...                        ...       ...       ...       ...  ...  
2022-05-27 062072.01       NaN       NaN       NaN       NaN  NaN  
           062723.01 -0.066655  0.080518 -2.043083  1.112229  0.0  
           106486.01 -1.364655  0.804500 -0.378720 -3.119503  0.0  
           164471.02       NaN       NaN       NaN       NaN  NaN  
           178518.02  0.000000  0.023024 -0.337761  2.051161  0.0  

[10125180 rows x 10 columns]

In [21]:
df['label'] = np.reshape(dl.label, N*T)

In [23]:
df['label_resid'] = np.reshape(np.load('data/forward_week_resid_ret.npy'), N*T)

In [25]:
df.to_csv('data/fct_data.csv')

In [2]:
gBar = tqdm(
        range((len(dl.date) - 350)//50),
        colour="red",
        desc=f"{model.name} Inferencing & Predicting",
    )
pred_result = []
label_result = []
date_result = []
for g in gBar:
    model.reset_weight()
    model.refit()
    gBar.set_postfix({"Year": int(model.test_period[0][:4])})
    # model.train()
    pred, label, date = model.predict()
    pred_result+=pred
    label_result+=label
    date_result+=date
pred_result = pd.concat(pred_result,axis=1)
label_result = pd.concat(label_result, axis=1)
pred_result.columns, label_result.columns = date_result, date_result
pred_result.to_csv(model.save_path+'prediction'+f'/{model.name}.csv')
label_result.to_csv(model.save_path+'label'+f'/{model.name}.csv')

try Inferencing & Predicting:  31%|███▏      | 5/16 [03:42<08:09, 44.47s/it, Year=2011]


KeyboardInterrupt: 

In [3]:
pred_result = pd.read_csv(model.save_path+'prediction'+f'/{model.name}.csv')
label_result = pd.read_csv(model.save_path+'label'+f'/{model.name}.csv')
oos_prediction_R2 = 1 - np.nansum((pred_result - label_result)**2, axis=1)/ np.nansum(label_result**2, axis=1)

FileNotFoundError: [Errno 2] No such file or directory: 'results/prediction/try.csv'

In [2]:
dl.update_period(['2006-01-01', '2006-02-09'])

tensor(0.9993)